In [2]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.autograd import Variable  # http://taewan.kim/trans/pytorch/tutorial/blits/02_autograd/
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

import pickle
import optuna
import torch.optim as optim 
import joblib

In [4]:
with open('../data/crypto_currency.pickle', 'rb') as f:  # 'rb'는 binary로 읽겠다는 의미 (문자열이 아닌 객체를 읽어보겠다는 뜻)
    data = pickle.load(f)

In [5]:
bch = data['KRW-ETH'][['candle_date_time_kst', 'trade_price']]
bch.head()

,candle_date_time_kst,trade_price
17,2017-09-25T09:00:00,325000.0
16,2017-09-26T09:00:00,321500.0
15,2017-09-27T09:00:00,342500.0
14,2017-09-28T09:00:00,332500.0
13,2017-09-29T09:00:00,327500.0


In [6]:
bch.set_index('candle_date_time_kst', inplace=True)

In [7]:
def sliding_windows(data, seq_length):
    x = []
    y = []
    
    for i in range(len(data) - seq_length - 1):  # i는 총 699개임, seq_length는 20개, i는 예측 대상일이므로 1개가됨. 
        _x = data[i: (i + seq_length)]  # _x에는 길이가 20인 vector가 들어감
        _y = data[i + seq_length]
        x.append(_x)
        y.append(_y)
    return np.array(x), np.array(y)

In [9]:
sc = MinMaxScaler()
training_data = sc.fit_transform(bch[['trade_price']])

In [10]:
x, y = sliding_windows(training_data, 20)

In [122]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()  # 객체화 과정에서 가장 먼저 사용하는 함수(initialize), 메모리에 올릴 때 이 작업을 함
        self.num_classes = num_classes  # 함수의 인자들을 class가 인식할 수 있도록 self 키워드를 붙여서 다시 저장
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, 
                            num_layers = num_layers, batch_first = True)  # lstm이라는 layer가 괄호 속 인자들을 가지고 있는 pytorch의 LSTM 모듈을 가짐.
        
        self.fc = nn.Linear(hidden_size, num_classes)  # fully connected layer, num_classes는 1개
        # input_size는 1개가 20번씩 들어감 (20개의 일자), hidden_size는 cell state와 hidden state의 크기로 hyper param임 (보통 4의 배수를 사용)
        # num_layers는 layer들을 stacking하는 개수를 의미 
    
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))  # 첫 단계의 값이 없으므로 zero 행렬 이용 0으로 설정
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        # propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
#         h_out = h_out.view(-1, self.hidden_size)  # output은 매 단계마다 나오며, 그 중 맨 마지막 값만 출력
        h_out = h_out[-1, :, :].view(-1, self.hidden_size)
#         print(f'hidden = {self.hidden_size}')
#         print(f'seq_len = {seq_length}')
#         print(f'h shape = {h_out.shape}')
        
        out = self.fc(h_out)
        
        return out

In [123]:
seq_length = 20

In [124]:
def train_model(trial): 
  cfg = {'train_batch_size' : trial.suggest_categorical('train_batch_size',[16, 32, 64, 128, 256, 512]),
         'test_batch_size' : trial.suggest_categorical('test_batch_size',[16, 32, 64, 128, 256, 512]), 
         'num_epochs' : trial.suggest_int('n_epochs', 5, 50, 1), 
         'seed' : 0, 
         'save_model' : False, 
         'lr' : trial.suggest_loguniform('lr', 1e-3, 1e-2), 
         'seq_length' : trial.suggest_int('seq_length', 7, 30, 1),
         'num_layers' : trial.suggest_int('num_layers', 1, 3, 1),
         'hidden_size' : trial.suggest_categorical('hidden_size',[16, 32, 64, 128, 256, 512]),
         'input_size' : 1,
         'num_classes' : 1,
         'optimizer': trial.suggest_categorical('optimizer',[torch.optim.Adam])} 
         
  torch.manual_seed(cfg['seed']) 

  train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
  train_dl = DataLoader(train_ds, batch_size=cfg['train_batch_size'])

  test_ds = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
  test_dl = DataLoader(test_ds, batch_size=cfg['test_batch_size'])

  model = LSTM(num_classes=cfg['num_classes'], 
               input_size =cfg['input_size'], 
               hidden_size=cfg['hidden_size'], 
               num_layers=cfg['num_layers'], 
               seq_length=cfg['seq_length']
  )
  optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr']) 
  criterion = torch.nn.MSELoss()

  for epoch in range(1, cfg['num_epochs'] + 1):
    for xb, yb in train_dl:
        outputs = model(xb)
        optimizer.zero_grad()
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, loss: {loss.item()}')    
    
  if cfg['save_model']: 
    torch.save(model.state_dict(), "lstm_optuna.pt") 

  return loss.item()

In [125]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0, shuffle=False)

In [126]:
sampler = optuna.samplers.TPESampler() 

study = optuna.create_study(sampler=sampler, direction='maximize') 
study.optimize(train_model, n_trials=5) 
joblib.dump(study, '/content/gdrive/My Drive/Colab_Data/studies/mnist_optuna.pkl')

[I 2022-05-22 22:24:04,310] A new study created in memory with name: no-name-8e3947d6-dd56-42d9-8ca6-da213b93d23f
/home/kyle/.pyenv/versions/image_crawler/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)
[I 2022-05-22 22:24:06,603] Trial 0 finished with value: 0.0619717538356781 and parameters: {'train_batch_size': 64, 'test_batch_size': 512, 'n_epochs': 11, 'lr': 0.0071683185174127315, 'seq_length': 26, 'num_layers': 1, 'hidden_size': 64, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 0 with value: 0.0619717538356781.


Epoch: 10, loss: 0.08626867085695267


/home/kyle/.pyenv/versions/image_crawler/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)


Epoch: 10, loss: 0.03725602850317955


[I 2022-05-22 22:24:09,781] Trial 1 finished with value: 0.005028170067816973 and parameters: {'train_batch_size': 128, 'test_batch_size': 512, 'n_epochs': 17, 'lr': 0.009853268519878123, 'seq_length': 8, 'num_layers': 1, 'hidden_size': 64, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 0 with value: 0.0619717538356781.
/home/kyle/.pyenv/versions/image_crawler/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)
[I 2022-05-22 22:24:14,088] Trial 2 finished with value: 0.06282902508974075 and parameters: {'train_batch_size': 16, 'test_batch_size': 32, 'n_epochs': 9, 'lr': 0.0010933762086160212, 'seq_length': 18, 'num_layers': 1, 'hidden_size': 16, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 2 with value: 0.06282902508974075.
/home/kyle/

Epoch: 10, loss: 0.646763801574707
Epoch: 20, loss: 0.058330684900283813
Epoch: 30, loss: 0.043867215514183044


[I 2022-05-22 22:25:55,373] Trial 3 finished with value: 0.05614763870835304 and parameters: {'train_batch_size': 128, 'test_batch_size': 32, 'n_epochs': 38, 'lr': 0.0026867991671455853, 'seq_length': 9, 'num_layers': 3, 'hidden_size': 256, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 2 with value: 0.06282902508974075.
/home/kyle/.pyenv/versions/image_crawler/lib/python3.8/site-packages/optuna/distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)


Epoch: 10, loss: 0.051313694566488266
Epoch: 20, loss: 0.014670992270112038


[I 2022-05-22 22:27:01,646] Trial 4 finished with value: 0.047387246042490005 and parameters: {'train_batch_size': 128, 'test_batch_size': 32, 'n_epochs': 27, 'lr': 0.004685448735516767, 'seq_length': 25, 'num_layers': 3, 'hidden_size': 256, 'optimizer': <class 'torch.optim.adam.Adam'>}. Best is trial 2 with value: 0.06282902508974075.


FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/Colab_Data/studies/mnist_optuna.pkl'